# SECTION 1 - Preparing for analysis

In [1]:
### Impor packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import sys, re, spacy
from textblob import TextBlob
import textstat
import spacy
nlp=spacy.load('en_core_web_md')
from collections import Counter

In [2]:
### creating ticker list
ticker_list = ['NVDA.p','MA.p', 'KO.p', 'TD.p', 'TSLA.p', 'GS.p', 'JNJ.p', 'SBUX.p', 'NVO.p', 'INTU.p', 'NFLX.p', 'BX.p', 'LLY.p', 'PEP.p', 'EBAY.p', 'SAP.p', 'FB.p', 'ABBV.p', 'CHTR.p', 'MRK.p', 'BMY.p', 'ADBE.p', 'ORCL.p', 'F.p', 'NKE.p', 'BKNG.p', 'GM.p', 'UPS.p', 'T.p', 'AMD.p', 'CMCSA.p', 'DIS.p', 'MDT.p', 'SNY.p', 'INTC.p', 'MS.p', 'HON.p', 'DUK.p', 'BLK.p', 'TGT.p', 'CSCO.p', 'TMUS.p', 'UNH.p', 'WMT.p', 'AXP.p',  'FDX.p', 'MCD.p', 'GOOG.p', 'ASML.p', 'V.p', 'BAC.p', 'RY.p', 'AMT.p', 'WFC.p', 'XOM.p', 'MET.p', 'QCOM.p', 'PM.p', 'CVS.p', 'MSFT.p', 'AMZN.p', 'IBM.p', 'VZ.p', 'ACN.p', 'C.p', 'HD.p', 'NVS.p', 'ISRG.p', 'AZN.p', 'CRM.p', 'PG.p', 'AAPL.p', 'TMO.p', 'TSM.p', 'EL.p', 'DHR.p', 'TXN.p', 'JPM.p', 'PFE.p', 'SE.p', 'CVX.p', 'NEE.p', 'AMGN.p', 'COST.p', 'BA.p', 'ABT.p', 'MCO.p', 'LOW.p', 'LMT.p', 'UNP.p']
len(ticker_list)

90

In [3]:
tl4 = ticker_list[30:40]
print(f"Detta är våra tickers, {tl4}")

Detta är våra tickers, ['CMCSA.p', 'DIS.p', 'MDT.p', 'SNY.p', 'INTC.p', 'MS.p', 'HON.p', 'DUK.p', 'BLK.p', 'TGT.p']


In [4]:
## Getting Loughran & McDonald (LM) Wordlists
LM_negative=pd.read_excel('./shared/Lecture 3 Text Analysis/LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name='Negative', header=None)
LM_positive=pd.read_excel('./shared/Lecture 3 Text Analysis/LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name='Positive', header=None)
LM_uncertainty=pd.read_excel('./shared/Lecture 3 Text Analysis/LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name='Uncertainty', header=None)

In [5]:
#preparing wordlists
LM_positive=LM_positive.T.values.tolist()[0]
LM_negative=LM_negative.T.values.tolist()[0]
LM_uncertainty=LM_uncertainty.T.values.tolist()[0]
len(LM_positive), len(LM_negative), len(LM_uncertainty)

(354, 2355, 297)

In [6]:
## defining process function that removes punctuation, stop words and spaces
def process_spacy_doc(text):
    # Tokenize
    # Remove stop words, spaces, punctuation
    # Lemmatize
    # Convert to lower case
    words=[
        word.lemma_.lower()                 # Lemmatize and lower case
        for word in nlp(text)          # Tokenize
        if not (
            word.is_space                   # Remove spaces
            or word.is_stop                 # Remove stop words
            or word.is_punct                # Remove punctuation
        )
    ]
    return words

In [7]:
LM_positive=process_spacy_doc(' '.join(LM_positive))
LM_negative=process_spacy_doc(' '.join(LM_negative))
LM_uncertainty=process_spacy_doc(' '.join(LM_uncertainty))
len(LM_positive), len(LM_negative), len(LM_uncertainty)

(354, 2353, 288)

In [8]:
LM_positive=set(process_spacy_doc(' '.join(LM_positive)))
LM_negative=set(process_spacy_doc(' '.join(LM_negative)))
LM_uncertainty=set(process_spacy_doc(' '.join(LM_uncertainty)))
len(LM_positive), len(LM_negative), len(LM_uncertainty)

(239, 1341, 194)

In [9]:
def score_transcript(text):
    # Running the function we created earlier that remove stopwords, spaces and so on... 
    words = process_spacy_doc(text)

        
    # This is the same as above as well. 
    # We the save the amount of words for later. This will show us how many % of the words are e.g. positive. 
    number_of_words=len(words)
    
    counts=Counter(words)

    keys=set(counts.keys())

    pos = round((sum([counts[k] for k in (keys & LM_positive)]) / number_of_words), 4)
    neg = round((sum([counts[k] for k in (keys & LM_negative)]) / number_of_words), 4)
    unc = round((sum([counts[k] for k in (keys & LM_uncertainty)]) / number_of_words), 4)
    
    return(pos, neg, unc)

# SECTION 2 - Sentiment analysis & CSV.

In [77]:
## Change ticker to be loaded 0-9
tl = tl4[9]
size = len(tl)
ticker_name = tl[:size - 2]
print(ticker_name)

TGT


In [78]:
## Load transcripts from ticker variable "tl"
transcripts = pickle.load(open( str("./shared/Transcripts/"+tl), "rb" ) )

In [79]:
## transctipts to list
transcripts_texts = []

# times to list
times = []
fqtr = []

for t in transcripts:
    text = ''
    for speech in t['transcript']:
        text = str(text + ' '+ speech['speech'][0])

    transcripts_texts.append(text)
    
    times.append(t['time'])
    
    fqtr.append(t['quarter'])
    
len(transcripts_texts), len(times), len(fqtr)

(72, 72, 72)

In [80]:
## Testing if transcripts are to long and then shortening it
for i in range(len(transcripts_texts)):    
    if (len(transcripts_texts[i]) > 80000):
        print()
        print('TOO LONG .... going to make it shorter')
        print(i, len(transcripts_texts[i]))
        transcripts_texts[i] = transcripts_texts[i][:80000]


TOO LONG .... going to make it shorter
0 117440

TOO LONG .... going to make it shorter
8 87841

TOO LONG .... going to make it shorter
12 81330

TOO LONG .... going to make it shorter
16 96970

TOO LONG .... going to make it shorter
24 127113

TOO LONG .... going to make it shorter
31 128305

TOO LONG .... going to make it shorter
48 103221


In [81]:
# Creating a dataframe to save all scores.
sentiment_scores = pd.DataFrame(columns=['pos', 'neg', 'unc'])

# Loop through all texts 
for texts in transcripts_texts:
    # Calls our scoring function and gets 3 scores back
    scores = score_transcript(texts)
    
    # This creates a new dataframe with one line of 3 scores 
    # and adds it to the bottom of our dataframe that stores all scores
    sentiment_scores = pd.concat([sentiment_scores, pd.DataFrame([np.array(scores)], columns=['pos', 'neg', 'unc'])]).reset_index(drop=True)

In [82]:
sentiment_scores.index = pd.to_datetime(times) # <-- THIS IS WHERE YOU HAVE CHANGE THE LIST OF TIMES

# This is new
sentiment_scores['dates'] = pd.to_datetime(times) # <-- I've added the dates to enable you to merge on='dates' later on.
# add the ticker 
sentiment_scores['ticker'] = ticker_name

sentiment_scores['fqtr'] = fqtr

# Lets also reverse the order so that old is first. Makes it easier to merge. 
# https://stackoverflow.com/a/3940137
sentiment_scores = sentiment_scores[::-1]

sentiment_scores

,pos,neg,unc,dates,ticker,fqtr
2006-02-16 23:33:06,0.0422,0.0196,0.0103,2006-02-16 23:33:06,TGT,4
2006-05-15 18:19:16,0.0350,0.0243,0.0099,2006-05-15 18:19:16,TGT,1
2006-08-10 18:15:11,0.0396,0.0253,0.0141,2006-08-10 18:15:11,TGT,2
2006-11-14 22:00:09,0.0388,0.0243,0.0125,2006-11-14 22:00:09,TGT,3
2007-02-27 21:26:18,0.0442,0.0147,0.0108,2007-02-27 21:26:18,TGT,4
...,...,...,...,...,...,...
2020-03-04 00:42:12,0.0486,0.0195,0.0072,2020-03-04 00:42:12,TGT,4
2020-05-20 19:01:55,0.0367,0.0325,0.0097,2020-05-20 19:01:55,TGT,1
2020-08-19 17:58:10,0.0416,0.0205,0.0133,2020-08-19 17:58:10,TGT,2
2020-11-18 18:46:16,0.0428,0.0194,0.0109,2020-11-18 18:46:16,TGT,3


In [83]:
# save as csv
sentiment_scores.to_csv("./csv/"+ticker_name+".csv")
print(f'Transcripts for ticker "{ticker_name}" is analysed and saved as csv')

Transcripts for ticker "TGT" is analysed and saved as csv


# Section 3 - importing fundamental data from WRDS

In [4]:
import wrds
## Connecting to wrds
db = wrds.Connection()carl

Enter your WRDS username [1001150000]: gusborcag
Enter your password: ····················


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
tickerlist = [x[:-2] for x in tl4]
tickerlist

['CMCSA', 'DIS', 'MDT', 'SNY', 'INTC', 'MS', 'HON', 'DUK', 'BLK', 'TGT']

In [22]:
### niq = net income quarterly
fundq = db.raw_sql("""select datadate, fyearq, fqtr, tic, niq 
                        
                        from comp.fundq
                        
                        where tic = 'CMCSA'
                        and datadate>='01/01/2005'
                        
                        or tic = 'DIS'
                        and datadate>='01/01/2005'
                        
                        or tic = 'MDT'
                        and datadate>='01/01/2005'
                        
                        or tic = 'SNY'
                        and datadate>='01/01/2005'
                        
                        or tic = 'INTC'
                        and datadate>='01/01/2005'
                        
                        or tic = 'MS'
                        and datadate>='01/01/2005'
                        
                        or tic = 'HON'
                        and datadate>='01/01/2005'
                        
                        or tic = 'DUK'
                        and datadate>='01/01/2005'
                        
                        or tic = 'BLK'
                        and datadate>='01/01/2005'
                        
                        or tic = 'TGT'
                        and datadate>='01/01/2005'""",
                   
                     date_cols=['datadate'])

fundq

,datadate,fyearq,fqtr,tic,niq
0,2016-09-30,2016.0,3.0,HON,1240.0
1,2017-09-30,2017.0,3.0,HON,1346.0
2,2018-09-30,2018.0,3.0,HON,2339.0
3,2005-03-31,2005.0,1.0,HON,358.0
4,2005-06-30,2005.0,2.0,HON,302.0
...,...,...,...,...,...
659,2020-09-30,2020.0,3.0,BLK,1364.0
660,2020-12-31,2020.0,4.0,BLK,1548.0
661,2021-03-31,2021.0,1.0,BLK,1199.0
662,2021-06-30,2021.0,2.0,BLK,1378.0


In [23]:
fundq.tic.unique()

array(['HON', 'CMCSA', 'TGT', 'DIS', 'DUK', 'INTC', 'MDT', 'MS', 'SNY',
       'BLK'], dtype=object)

In [24]:
## no decimals for quarter and year
fundq['fyearq'] = fundq['fyearq'].astype(int)
fundq['fqtr'] = fundq['fqtr'].astype(int)

In [25]:
fundq

,datadate,fyearq,fqtr,tic,niq
0,2016-09-30,2016,3,HON,1240.0
1,2017-09-30,2017,3,HON,1346.0
2,2018-09-30,2018,3,HON,2339.0
3,2005-03-31,2005,1,HON,358.0
4,2005-06-30,2005,2,HON,302.0
...,...,...,...,...,...
659,2020-09-30,2020,3,BLK,1364.0
660,2020-12-31,2020,4,BLK,1548.0
661,2021-03-31,2021,1,BLK,1199.0
662,2021-06-30,2021,2,BLK,1378.0


# Section 4 -  Loading CSVs to one dataframe

In [13]:
## Loading all csvs to one data frame
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
import glob

path = r"./csv/"
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

sentimentdf = pd.concat(li, axis=0, ignore_index=True)

In [14]:
sentimentdf

,Unnamed: 0,pos,neg,unc,dates,ticker,fqtr
0,2007-08-22 00:05:43,0.0384,0.0219,0.0214,2007-08-22 00:05:43,MDT,1
1,2008-02-19 19:47:11,0.0373,0.0257,0.0103,2008-02-19 19:47:11,MDT,3
2,2008-05-20 20:10:11,0.0487,0.0253,0.0150,2008-05-20 20:10:11,MDT,4
3,2008-08-19 18:16:28,0.0463,0.0171,0.0120,2008-08-19 18:16:28,MDT,1
4,2009-02-17 19:11:28,0.0463,0.0247,0.0128,2009-02-17 19:11:28,MDT,3
...,...,...,...,...,...,...,...
708,2020-02-05 04:09:43,0.0293,0.0239,0.0152,2020-02-05 04:09:43,DIS,1
709,2020-05-06 04:59:34,0.0209,0.0483,0.0162,2020-05-06 04:59:34,DIS,2
710,2020-08-05 03:43:07,0.0441,0.0337,0.0136,2020-08-05 03:43:07,DIS,3
711,2020-11-13 02:39:12,0.0412,0.0339,0.0139,2020-11-13 02:39:12,DIS,4


In [15]:
## adding a year column
sentimentdf['fyearq'] = [str(x)[0:4] for x in sentimentdf['dates']]

In [16]:
## preparing for merge
sentimentdf['fyearq'] = sentimentdf['fyearq'].astype(int)
sentimentdf.rename(columns={'ticker': 'tic'}, inplace=True)

# Section 5 - Merging dataframes

In [26]:
## merging dataframes
dfmerged= pd.merge(sentimentdf, fundq)

In [27]:
### Sorting data in the right order
dfmerged.sort_values(by=['tic','fyearq','fqtr'], inplace=True,
               ascending = [True, True, True])

In [28]:
# Reset index
dfmerged.reset_index(drop=True, inplace=True)

In [29]:
# Dropping unneccesary columns
dfmerged = dfmerged.drop(columns=['dates', 'datadate', 'Unnamed: 0'])

In [30]:
dfmerged

,pos,neg,unc,tic,fqtr,fyearq,niq
0,0.0405,0.0294,0.0230,BLK,3,2007,255.200
1,0.0369,0.0302,0.0272,BLK,1,2008,241.671
2,0.0368,0.0325,0.0244,BLK,2,2008,274.058
3,0.0391,0.0359,0.0296,BLK,3,2008,217.726
4,0.0391,0.0260,0.0235,BLK,4,2008,52.545
...,...,...,...,...,...,...,...
540,0.0512,0.0097,0.0057,TGT,4,2019,834.000
541,0.0367,0.0325,0.0097,TGT,1,2020,284.000
542,0.0416,0.0205,0.0133,TGT,2,2020,1690.000
543,0.0428,0.0194,0.0109,TGT,3,2020,1014.000


KeyError: 'tic,fyearq'

365

10.15